In [1]:
import pandas as pd
import numpy as np
from glob import glob
import gcsfs
pd.set_option('display.width', 1000)

### Find datasets which have a new version since a prior date
 - Specify the catalog date to compare to current catalog
 - Specify catalog type ('-noQC' or '') 

In [6]:
fs = gcsfs.GCSFileSystem(token='anon',access='read_only',cache_timeout=-1)

date = '20210120'
cat_type = '-noQC'

# find closest catalog to specified date:
files = fs.glob(f'gs://cmip6/CMIP6/old_catalogs/pangeo-cmip6-*{cat_type}.csv.gz')
dates = [int(s.split('cmip6-')[-1].split(cat_type)[0]) for s in files]
prior_date = str(min(dates, key=lambda x:abs(x-int(date))))
prior_date

'20210120'

In [7]:
# Read the catalogs
df_old = pd.read_csv(f'https://cmip6.storage.googleapis.com/CMIP6/old_catalogs/pangeo-cmip6-{prior_date}{cat_type}.csv.gz', dtype='unicode')
df = pd.read_csv(f'https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores{cat_type}.csv', dtype='unicode')

In [8]:
dfn = pd.merge(df_old, df[['zstore','version']], on='zstore', how='left', sort=False, suffixes=('_old', '_new'))
df_replaced = dfn[dfn.version_old != dfn.version_new]
df_replaced = df_replaced[~(df_replaced.version_new.isnull())]

In [9]:
df_replaced[['zstore', 'version_old','version_new']].style 

,zstore,version_old,version_new
45349,gs://cmip6/CMIP/E3SM-Project/E3SM-1-0/historical/r4i1p1f1/Omon/tos/gr/,20200129,20200813
46109,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r1i1p1f1/Amon/evspsbl/gr/,20200217,20201122
46190,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r2i1p1f1/Amon/evspsbl/gr/,20200217,20201122
46249,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r3i1p1f1/Amon/evspsbl/gr/,20200217,20201122
46286,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r3i1p1f1/Omon/tos/gn/,20200217,20200919
46347,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/1pctCO2/r1i1p1f1/Amon/evspsbl/gr/,20190702,20201122
46426,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/abrupt-4xCO2/r1i1p1f1/Amon/evspsbl/gr/,20190702,20201122
46628,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r1i1p1f1/Amon/evspsbl/gr/,20190605,20201122
46760,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r2i1p1f1/Amon/evspsbl/gr/,20200226,20201122
46817,gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r2i1p1f1/Omon/tos/gn/,20200226,20200919
